In [ ]:
su

# Python Project - Samuele Ceol 
A case study on the correlation between the labour market and suicide rates in Europe

In [593]:
import matplotlib.pyplot as plt 
import numpy as np 
import pandas as pd
import seaborn as sns 

# OECD Data - Description

For the OECD we have three distinct files, one for males, one for females and one aggregate (M/F).

The dataset contains only data related to european countries.

No age group distinction is provided.

The column LOCATION identifies the interested country with a unique three letter abbreviation.

Suicide rates are indicated as nr of suicides per 100.000 people.

TODO - Flag codes?

In [594]:
suicide_tot = pd.read_csv('./source/OECD_suicides_total.csv')
suicide_male = pd.read_csv('./source/OECD_suicides_male.csv')
suicide_female = pd.read_csv('./source/OECD_suicides_female.csv')

In [595]:
suicide_tot.shape

(1955, 8)

In [596]:
suicide_tot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1955 entries, 0 to 1954
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   LOCATION    1955 non-null   object 
 1   INDICATOR   1955 non-null   object 
 2   SUBJECT     1955 non-null   object 
 3   MEASURE     1955 non-null   object 
 4   FREQUENCY   1955 non-null   object 
 5   TIME        1955 non-null   int64  
 6   Value       1955 non-null   float64
 7   Flag Codes  98 non-null     object 
dtypes: float64(1), int64(1), object(6)
memory usage: 122.3+ KB


In [597]:
suicide_tot.head()

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
0,AUS,SUICIDE,TOT,100000PER,A,1960,13.1,NaN
1,AUS,SUICIDE,TOT,100000PER,A,1961,14.5,NaN
2,AUS,SUICIDE,TOT,100000PER,A,1962,16.9,NaN
3,AUS,SUICIDE,TOT,100000PER,A,1963,19.2,NaN
4,AUS,SUICIDE,TOT,100000PER,A,1964,17.8,NaN


# OECD Data - Required actions

We would like to join this three different entities into a single dataframe.

We start by adding a column that identifies which category (Male/Female/Total) the dataframe refers to.


In [598]:
suicide_tot['Sex']     = 'total'
suicide_male['Sex']    = 'male'
suicide_female['Sex']  = 'female'

We then proceed to concatenate the three dataframes.

In [599]:
OECD_suicide = pd.concat([suicide_tot, suicide_male, suicide_female])

We can discard the INDICATOR, SUBJECT, MEASURE and FREQUENCY columns since they are only descriptive and not needed for our analysis

In [600]:
OECD_suicide = OECD_suicide.drop(['INDICATOR', 'SUBJECT', 'MEASURE', 'FREQUENCY', 'Flag Codes'], axis=1)

To join the OECD dataset with the one provided by the World Health Organization, we want to change the three letters identifiers to the full country names.
To do that, we use the pycountry library.

In [601]:
import pycountry

OECD_suicide['LOCATION'] = OECD_suicide['LOCATION'].apply(
    lambda x: pycountry.countries.get(alpha_3=x).name 
    if len(x) == 3 
    else pycountry.countries.get(alpha_2=x).name
)

We can now filter out the countries in the dataset that are not in Europe

TODO - find out if there is a better way to get the list of european countries

In [602]:
euro_countries = ['Russian Federation','Ukraine','France','Spain','Sweden','Norway','Germany','Finland','Poland','Italy','United Kingdom','Romania','Belarus','Kazakhstan','Greece','Bulgaria','Iceland','Hungary','Portugal','Austria','Czechia','Serbia','Ireland','Lithuania','Latvia','Croatia','Bosnia and Herzegovina','Slovakia','Estonia','Denmark','Switzerland','Netherlands','Moldova','Belgium','Armenia','Albania','North Macedonia','Turkey','Slovenia','Montenegro','Kosovo','Azerbaijan','Cyprus','Luxembourg','Georgia','Andorra','Malta','Liechtenstein','San Marino','Monaco','Vatican City']

print('Initial nr of countries: ')
print(OECD_suicide['LOCATION'].nunique())

OECD_suicide = OECD_suicide[OECD_suicide['LOCATION'].isin(euro_countries)]

print('Final nr of countries: ')
print(OECD_suicide['LOCATION'].nunique())

#List of european countries that are not present in this dataset
print('European countries not present in this dataset: ')
print([x for x in euro_countries if x not in OECD_suicide['LOCATION'].unique().tolist()])

Initial nr of countries: 
41
Final nr of countries: 
28
European countries not present in this dataset: 
['Ukraine', 'Romania', 'Belarus', 'Kazakhstan', 'Bulgaria', 'Serbia', 'Croatia', 'Bosnia and Herzegovina', 'Moldova', 'Armenia', 'Albania', 'North Macedonia', 'Montenegro', 'Kosovo', 'Azerbaijan', 'Cyprus', 'Georgia', 'Andorra', 'Malta', 'Liechtenstein', 'San Marino', 'Monaco', 'Vatican City']


We can finally rename the remaining columns and see how the dataframe looks like.

TODO - Maintain the groupby even though we don't have duplicates

In [603]:
OECD_suicide.head()

,LOCATION,TIME,Value,Sex
57,Austria,1960,24.2,total
58,Austria,1961,23.2,total
59,Austria,1962,23.8,total
60,Austria,1963,23.0,total
61,Austria,1964,24.1,total


In [604]:
OECD_suicide.columns = ['country', 'year', 'suicides_no', 'sex']
OECD_suicide = OECD_suicide.sort_values(['country', 'year','sex']).reset_index(drop = True)

In [605]:
OECD_suicide.head()

,country,year,suicides_no,sex
0,Austria,1960,14.7,female
1,Austria,1960,36.6,male
2,Austria,1960,24.2,total
3,Austria,1961,13.7,female
4,Austria,1961,35.5,male


# WHO Data - Description

For the data provided by the World Health Organization we have a single file containing data for males and females with a further division by age group.

Suicide values are stored as totals unlike the previous dataframe where they were stored in relation to nr of suicides per 100.000.

The dataframe also contains information related to the total population of the country in the given year.


In [606]:
WHO_suicide = pd.read_csv('./source/WHO_suicides_aggregate.csv')

In [607]:
WHO_suicide.shape

(43776, 6)

In [608]:
WHO_suicide.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43776 entries, 0 to 43775
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   country      43776 non-null  object 
 1   year         43776 non-null  int64  
 2   sex          43776 non-null  object 
 3   age          43776 non-null  object 
 4   suicides_no  41520 non-null  float64
 5   population   38316 non-null  float64
dtypes: float64(2), int64(1), object(3)
memory usage: 2.0+ MB


In [609]:
WHO_suicide.head()

,country,year,sex,age,suicides_no,population
0,Albania,1985,female,15-24 years,NaN,277900.0
1,Albania,1985,female,25-34 years,NaN,246800.0
2,Albania,1985,female,35-54 years,NaN,267500.0
3,Albania,1985,female,5-14 years,NaN,298300.0
4,Albania,1985,female,55-74 years,NaN,138700.0


# WHO Data - Required actions

Just like for the OECD data, we filter out the countries that are not in Europe.

In [610]:
print('Initial nr of countries: ')
print(WHO_suicide['country'].nunique())

WHO_suicide = WHO_suicide[WHO_suicide['country'].isin(euro_countries)]

print('Final nr of countries: ')
print(WHO_suicide['country'].nunique())

#List of european countries that are not present in this dataset
print('European countries not present in this dataset: ')
print([x for x in euro_countries if x not in WHO_suicide['country'].unique().tolist()])

Initial nr of countries: 
141
Final nr of countries: 
44
European countries not present in this dataset: 
['Czechia', 'Moldova', 'North Macedonia', 'Kosovo', 'Andorra', 'Liechtenstein', 'Vatican City']


We can also drop the rows in which suicide data is not present.

In [611]:
WHO_suicide = WHO_suicide[WHO_suicide['suicides_no'].notna()]

For our analysis, we are not considering the differences between age groups.

Because of this reason, we can drop the 'age' column and sum population and suicide numbers.


In [612]:
WHO_suicide = WHO_suicide.drop('age', axis=1).groupby(['country', 'year', 'sex']).sum().unstack()

Since the dataframe only has a division between males and females, we want to add a new category for the total values which contains the sum for both nr of suicides between males and females and value of the total population.

To facilitate this process, we compute the totals for suicides numbers and population in two different variables and then merge them.

We then flatten the hierarchical index


In [613]:
WHO_suicide_mf      = WHO_suicide['suicides_no'].assign(total=WHO_suicide['suicides_no'].sum(1)).stack(level='sex')
WHO_population_mf   = WHO_suicide['population'].assign(total=WHO_suicide['population'].sum(1)).stack(level='sex')

WHO_suicide = pd.DataFrame(
    pd.concat(
        [WHO_suicide_mf, WHO_population_mf], 
        axis=1
    ).to_records()).rename(columns = {'0':'suicides_no', '1':'population'}
) 

Before converting the suicide nr to suicide rates, we should first address the few rows with missing population data.

My initial idea was to use The World Bank census data in order to fill in the gaps in the poulation numbers for the WHO dataset

The following (now commented) code, was used to drop the columns were not needed, pivot longer the year columns and filter out the non european countries.

In [614]:
'''
census_tot      = pd.read_csv('./source/census_total.csv').drop(['Country Code', 'Indicator Code'], axis=1)
census_male     = pd.read_csv('./source/census_male.csv').drop(['Country Code', 'Indicator Code'], axis=1)
census_female   = pd.read_csv('./source/census_female.csv').drop(['Country Code', 'Indicator Code'], axis=1)

census = pd.concat([census_tot, census_male, census_female])

census = pd.melt(
    census, 
    id_vars = ['Country Name', 'Indicator Name'], 
    value_vars = [str(i).zfill(4) for i in range(1960,2020)], 
    var_name='year', 
    value_name='population'
).sort_values(
    ['Country Name', 'year', 'Indicator Name']
)

census = census[census['Country Name'].isin(euro_countries)]
census
'''

"\ncensus_tot      = pd.read_csv('./source/census_total.csv').drop(['Country Code', 'Indicator Code'], axis=1)\ncensus_male     = pd.read_csv('./source/census_male.csv').drop(['Country Code', 'Indicator Code'], axis=1)\ncensus_female   = pd.read_csv('./source/census_female.csv').drop(['Country Code', 'Indicator Code'], axis=1)\n\ncensus = pd.concat([census_tot, census_male, census_female])\n\ncensus = pd.melt(\n    census, \n    id_vars = ['Country Name', 'Indicator Name'], \n    value_vars = [str(i).zfill(4) for i in range(1960,2020)], \n    var_name='year', \n    value_name='population'\n).sort_values(\n    ['Country Name', 'year', 'Indicator Name']\n)\n\ncensus = census[census['Country Name'].isin(euro_countries)]\ncensus\n"

This solution was not implemented because I later realized that:

    - Only 9 rows, out of the total 3759, were missing this data
    - The two interested countries (Monaco and San Marino) were missing this data also in the other datasets

Given the small amount of interested rows, we can drop them without compromising the value of our data.

In [615]:
WHO_suicide = WHO_suicide[WHO_suicide['population'] != 0]

We can now convert the suicide values to suicides per 100.000 and then drop the population column.

Suicide rate = (Nr of suicides / population) * 100.000

In [616]:
WHO_suicide['suicides_no'] = ((WHO_suicide['suicides_no'] / WHO_suicide['population']) * 100000).round(1)
WHO_suicide = WHO_suicide.drop('population', axis=1)

In [617]:
WHO_suicide.head()

,country,year,sex,suicides_no
0,Albania,1987,female,1.9
1,Albania,1987,male,3.4
2,Albania,1987,total,2.7
3,Albania,1988,female,1.6
4,Albania,1988,male,2.9


# Suicide data - Merging the two datasets

We are now ready to combine the data coming from the two datasets

In case we have rows with the same country, year and sex, we keep the OECD data (which was already provided in the format of suicides per 100.000 people)

In [618]:
suicide = pd.concat([OECD_suicide, WHO_suicide]).drop_duplicates(subset=['country', 'year', 'sex'], keep='first').sort_values(['country', 'year','sex']).reset_index(drop=True)

print('Shape of OECD data: ' + str(OECD_suicide.shape))
print('Shape of WHO data: ' + str(WHO_suicide.shape))
print('Shape of final suicide dataset: ' + str(suicide.shape))

Shape of OECD data: (4002, 4)
Shape of WHO data: (3750, 4)
Shape of final suicide dataset: (5187, 4)


In [619]:
suicide

,country,year,suicides_no,sex
0,Albania,1987,1.9,female
1,Albania,1987,3.4,male
2,Albania,1987,2.7,total
3,Albania,1988,1.6,female
4,Albania,1988,2.9,male
...,...,...,...,...
5182,United Kingdom,2015,11.6,male
5183,United Kingdom,2015,7.5,total
5184,United Kingdom,2016,3.3,female
5185,United Kingdom,2016,11.4,male
